# Deriving cancer gene's from expression data

See if it's possible to train a deep neural network tumor/normal binary classifier using just the Toil TCGA, TARGET and GTEX expression datasets:

https://xenabrowser.net/datapages/?host=https://toil.xenahubs.net)

In [1]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import keras

# fix random seed for reproducibility
np.random.seed(42)

/home/brad/anaconda3/envs/xenas/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
/home/brad/anaconda3/envs/xenas/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
#"""
#Detect if there are GPU's on this box and if so use them politely. If not use the CPU.
#"""
#import re
#import subprocess
#import keras.backend.tensorflow_backend
#
#
#if "CUDA_HOME" in os.environ:
#    # Pick a GPU that isn't being used and then only use the memory we need (allow_growth)
#    print("Using GPU")
#    def get_session():
#        return tf.Session(config=tf.ConfigProto(log_device_placement=True, allow_soft_placement=True,
#                                                gpu_options=tf.GPUOptions(allow_growth=True)))
#    keras.backend.tensorflow_backend.set_session(get_session())
#
#    utilization = re.findall(r"Utilization.*?Gpu.*?(\d+).*?Memory.*?(\d+)",
#                             subprocess.check_output(["nvidia-smi", "-q"]), 
#                             flags=re.MULTILINE | re.DOTALL)
#    print("GPU Utilization", utilization)
#
#    if ('0', '0') in utilization:
#        print("Using GPU Device:", utilization.index(('0', '0')))
#        os.environ["CUDA_VISIBLE_DEVICES"] = str(utilization.index(('0', '0')))
#        os.environ["CUDA_DEVICE_ORDER"]  = "PCI_BUS_ID"  # To ensure the index matches
#    else:
#        print("All GPUs in Use")
#        exit    
#else:
#    print("Using CPU")

In [3]:
from keras.utils.io_utils import HDF5Matrix
import h5py

input_file = "data/tumor_normal.h5"

with h5py.File(input_file, "r") as f:
    print("Datasets:", list(f.keys()))
    
# Use a subset of the data if debugging
#if os.getenv("DEBUG", "True") == "True":
#    X_train = HDF5Matrix(input_file, "X_train", start=0, end=1000)
#    X_test = HDF5Matrix(input_file, "X_test", start=0, end=200)
#    y_train = HDF5Matrix(input_file, "y_train", start=0, end=1000)
#    y_test = HDF5Matrix(input_file, "y_test", start=0, end=200)
#    print("Training on partial dataset")
#    epochs=1
#    batch_size=256
#else:
X_train = np.array(HDF5Matrix(input_file, "X_train"))
X_test  = np.array(HDF5Matrix(input_file, "X_test"))
y_train = np.array(HDF5Matrix(input_file, "y_train"))
y_test  = np.array(HDF5Matrix(input_file, "y_test"))
Y_
print("Training on full dataset")
epochs=8
batch_size=128
    
print("X_train.shape:", X_train.shape, "epochs:", epochs, "batch_size:", batch_size)
print("X_test.shape:", X_test.shape)

 

Datasets: ['X_test', 'X_train', 'class_labels', 'classes_test', 'classes_train', 'features', 'genes', 'labels', 'y_test', 'y_train']
Training on full dataset
X_train.shape: (15300, 60498) epochs: 8 batch_size: 128
X_test.shape: (3826, 60498)


In [ ]:
"""
Batch normalization with a sparse layer.

Achieves on Test:
3744/3826 [============================>.] - ETA: 0s(['loss', 'acc'], [0.52888519396363787, 0.89388395178336721])
"""
from keras.models import Model, Sequential
from keras.layers import InputLayer, Dense, BatchNormalization, Activation, Dropout
from keras.callbacks import EarlyStopping
from keras import regularizers

opt = keras.optimizers.rmsprop(lr=0.0000002, decay=1e-6)

classify = [
    InputLayer(input_shape=(X_train.shape[1],)),
    BatchNormalization(),
    
#    Dense(1000),
#     BatchNormalization(),
#    Dropout(.9),
#    Activation('selu'),
#
#    Dense(500),
#     BatchNormalization(),
#    Dropout(.7),
#    Activation('selu'),

#    Dense(200),
#     BatchNormalization(),
#    Dropout(.5),
#    Activation('selu'),

#    Dense(100),
#     BatchNormalization(),
#    Dropout(.8),
#    Activation('relu'),
#
#    Dense(100),
#     BatchNormalization(),
#    Dropout(.8),
#    Activation('relu'),
#    InputLayer(input_shape=(X_train.shape[1],)),
#    BatchNormalization(),
#    
#    Dense(1),
#    BatchNormalization(),
#    Activation('selu'),

#    Dense(1, activity_regularizer=regularizers.l1(1e-5)),
#    BatchNormalization(),
#    Activation('selu'),
    
    Dense(1),
    Activation('sigmoid')

    
#    Dense(200, activity_regularizer=regularizers.l1(.0001)),
#    BatchNormalization(),
#    Dropout(.6),
#    Activation('relu'),
    
#    Dense(1, kernel_regularizer=regularizers.l1(0.001)),
#    Activation('sigmoid')
]

model = Sequential(classify)

#model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])

callbacks=[EarlyStopping(monitor='acc', min_delta=0.05, patience=2, verbose=2, mode="max")]

#model.fit(X_train, y_train, epochs=epochs, batch_size=32, shuffle="batch", callbacks=callbacks, validation_data=(X_test, y_test))

#print(model.metrics_names, model.evaluate(X_test, y_test))

In [ ]:
model.fit(X_train, y_train, epochs=20, batch_size=32, shuffle="batch",validation_data=(X_test, y_test))
print(model.metrics_names, model.evaluate(X_test, y_test))


In [ ]:
import numpy as np
def RNA_mix(mix, naug):
    
    print( )
    samples=X_train.shape[0]
    rmix=np.random.rand()*(1-mix)
    print("mix ratio=", 1-rmix)
    for i in range(naug) :
        i0 = np.random.randint(samples)
        i1 = np.random.randint(samples)
        
        Xmix0 = X_train[i0,:] *(1-rmix) 
        Xmix1 = X_train[i1,:] *(rmix) 
        X_aug[i,:]=Xmix0 + Xmix1
        Y_aug[i]=y_train[i0]
       

naug=5000

X_aug=np.zeros((naug,X_train.shape[1]))
Y_aug=np.zeros((naug))


for k in range(1):
   for i in range(20):
      RNA_mix(.5, naug)   
      model.fit(X_aug, Y_aug, epochs=1, batch_size=128, shuffle="true",validation_data=(X_test, y_test))

   model.fit(X_train, y_train, epochs=3, batch_size=256, shuffle="true",validation_data=(X_test, y_test))



In [ ]:
"""
Save model and weights so we can copy them back from the GPU machine
to visualize and evaluate locally.
"""
model_json = model.to_json()
if not os.path.exists("models"):
    os.makedirs("models")
with open("models/model.json", "w") as f:
    f.write(model.to_json())

model.save_weights("models/weights.h5")
print("Saved to model.json and weights.json")